In [1]:
import cv2
import numpy as np
import os

In [2]:
dataset_path="./data/"
faceData=[]
labels=[]

classId=0
namesMap={}
offset=20


In [3]:
for f in os.listdir(dataset_path):
    print(f)

Sharmistha.npy
Souradeep.npy


In [4]:
for f in os.listdir(dataset_path):
    if f.endswith(".npy"):
        namesMap[classId]=f[:-4]
        dataItem=np.load(dataset_path + f)
        print(dataItem.shape)
        #x-values
        n=dataItem.shape[0]
        dataItem=dataItem.reshape((n,-1))
        print(dataItem.shape)
        m=dataItem.shape[0]

        faceData.append(dataItem)

        #Y-values
        target=classId*np.ones((m,))
        classId+=1
        labels.append(target)


(23, 100, 100, 3)
(23, 30000)
(29, 100, 100, 3)
(29, 30000)


In [5]:
print(faceData)
print(labels)

[array([[206, 205, 207, ..., 152, 161, 194],
       [215, 213, 225, ...,  97, 147, 199],
       [182, 183, 186, ...,  82, 117, 161],
       ...,
       [103, 124, 159, ...,  44,  49,  50],
       [ 96, 129, 164, ...,  36,  39,  37],
       [ 93, 128, 168, ...,  35,  41,  43]], dtype=uint8), array([[ 89,  88,  84, ...,  40,  77, 115],
       [186, 192, 184, ...,  41,  68, 109],
       [162, 159, 154, ...,  27,  63,  97],
       ...,
       [213, 212, 206, ..., 148, 113, 102],
       [214, 211, 206, ..., 141, 114, 107],
       [217, 215, 210, ..., 199, 177, 193]], dtype=uint8)]
[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])]


In [6]:
xtT=np.concatenate(faceData,axis=0)
yT=np.concatenate(labels,axis=0).reshape((-1,1))

print(xtT.shape)
print(yT.shape)


print(namesMap)

(52, 30000)
(52, 1)
{0: 'Sharmistha', 1: 'Souradeep'}


KNN

In [7]:

def dist(p, q):
    return np.sqrt(np.sum((p - q) ** 2))

def knn(X, y, xt, k=5):
    m = X.shape[0]
    dlist = []

    for i in range(m):
        d = dist(X[i], xt)
        dlist.append((d, y[i]))

    # Sort dlist based on distance
    dlist = sorted(dlist, key=lambda x: x[0])
    # Select the first k elements
    dlist = dlist[:k]

    # Separate distances and labels
    distances = np.array([item[0] for item in dlist])
    labels = np.array([item[1] for item in dlist])

    # Find the most common label
    unique_labels, counts = np.unique(labels, return_counts=True)
    idx = counts.argmax()
    pred = unique_labels[idx]

    return int(pred)


#predictions
cam=cv2.VideoCapture(0)
model=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

#camera object

while True:
    sucess,img=cam.read()
    if not sucess:
        print("Cannot read")
    
    faces=model.detectMultiScale(img,1.3,5)
    

   
    for f in faces:

        x,y,w,h=f
        

        #cropping the image I got
        cropped_image=img[y-offset:y+h+offset, x-offset:x+w+offset] #its just like slicing operation
        cropped_image=cv2.resize(cropped_image,(100,100))
        


        #predict using KNN
        classPredicted=knn(xtT,yT,cropped_image.flatten())
        #name
        namePredicted=namesMap[classPredicted]

        cv2.putText(img,namePredicted,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    
    cv2.imshow("Prediction_Window",img)
    key=cv2.waitKey(1)
    if(key==ord('q')):
        break

cam.release()
cv2.destroyAllWindows()


In [8]:
cam.release()
cv2.destroyAllWindows()
